In [177]:
import pandas as pd
import numpy as np
import datetime
import holidays



#csv reading
file_path = 'bee_data.csv'
df = pd.read_csv(file_path)
hour_forecast = 12
country = 'FI'
holiday_list = holidays.CountryHoliday(country)



#data transformation

#convering timestamp in nano sec to datetime and formatted_timestamp in hour basis. 
df['timestamp'] = pd.to_datetime(df['timestamp']//1e9, unit='s')
df['formatted_timestamp'] = df['timestamp'].dt.strftime("%Y-%m-%dT%H:00")
df['formatted_timestamp'] = pd.to_datetime(df['formatted_timestamp'])
df = df.sort_values(by='timestamp').reset_index()



#drop df_build = df_build.drop(columns = ['partition_0','partition_1'])

df = df.drop(columns = ['partition_0','partition_1'])


#creating 4 data frames: building_cmd control sent, building_read, actual value, live weather and forecast weather
df_build_cmd = df.loc[(df['measurement'] == 'building') & (df['name'].str.endswith('Cmd'))].dropna(axis=1).drop_duplicates()
df_build_read = df.loc[(df['measurement'] == 'building') & (df['name'].str.endswith('TE'))].dropna(axis=1).drop_duplicates()
df_weather_live = df.loc[(df['measurement'] != 'building') & (df['type']== 'live')].dropna(axis=1).drop_duplicates()
df_weather_forecast = df.loc[(df['measurement'] != 'building') & (df['type']== 'forecast')].dropna(axis=1).drop_duplicates(subset=['timestamp'])



# Calculate mean and standard deviation for each device as value is arranged in unit steps with median value
#merging df_build with median value column  and renmae 
#create new column with control command applied 
_ = df_build_read.groupby('deviceid') ['value'].unique().apply(np.median)
df_build_read = df_build_read.merge(_,left_on='deviceid',right_index=True)
df_build_read.rename(columns={'value_x': 'value', 'value_y': 'ref_value'}, inplace=True)
df_build_read['actual_ctrlCmd'] = df_build_read['value'] - df_build_read['ref_value']
df_build_read = df_build_read[['formatted_timestamp','actual_ctrlCmd']].drop_duplicates()



#_ =df_build.groupby('deviceid')['name'].unique().reset_index()
#_ = pd.concat([_['deviceid'], _['name'].apply(pd.Series)], axis=1)
#_.head()






In [173]:
x = df_weather_live.groupby('formatted_timestamp')['value'].mean().round(2).reset_index()
x['hour_week'] = ((x['formatted_timestamp'].dt.dayofweek) * 24+24) - (24 - x['formatted_timestamp'].dt.hour)
x['is_weekend'] = x['hour_week'].between(113,167).astype(int)
x['is_holiday'] = x['formatted_timestamp'].dt.date.isin(holiday_list).astype(int)
df_weather_forecast = df_weather_forecast[['formatted_timestamp','value']]
df_weather_forecast['12_hour_forecast'] = df_weather_forecast['value'].shift(-12)
df_weather_forecast = df_weather_forecast.drop(columns = ['value'])
x = pd.merge_ordered(x,df_weather_forecast,on='formatted_timestamp')
x = pd.merge_ordered(x,df_build_read,on='formatted_timestamp')
x = x.rename(columns={'value':'live_temperature'})
x['temp_diff'] = x['12_hour_forecast'] - x['live_temperature']

In [176]:
conditions = [
    (x['temp_diff'] <= -3),
    (x['temp_diff'] > -3) & (x['temp_diff'] <= -2),
    (x['temp_diff'] > -2) & (x['temp_diff'] <= -1),
    (x['temp_diff'] > -1) & (x['temp_diff'] <=  1),
    (x['temp_diff'] >  1) & (x['temp_diff'] <=  2),
    (x['temp_diff'] >  2) & (x['temp_diff'] <=  3),
    (x['temp_diff'] >  3)
    ]

# create a list of the values we want to assign for each condition
values = [3,2,1,0,-1,-2,-3]

# create a new column and use np.select to assign values to it using our lists as arguments
x['the_ctrl_Cmd'] = np.select(conditions, values)

condition = (x['is_weekend'] ==1) | (x['is_holiday'] == 1 )
new_value = -2
x.loc[condition,'the_ctrl_Cmd'] = new_value


# display updated DataFrame
x.dropna(subset=['temp_diff'], inplace=True)



In [178]:
x.to_csv('x_data.csv', index=False)